In [2]:
# Initial imports

import panel as pn
pn.extension('plotly')
pn.extension()
from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets

import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

import numpy as np

import hvplot.pandas

from datetime import date, timedelta

#pn.extension('plotly')

import plotly.express as px
import matplotlib.pyplot as plt
from pathlib import Path

#####################################################
#new Library used for project to perform regressions
#pip install SciPy ----- in Pyviz environment
from scipy import stats
####################################################

%matplotlib inline

In [3]:
# Load .env environment variables
# ensure both mapbox and alpaca keys are avilable in the env file
# ensure to install Alpaca INSIDE PyVix environemnt
load_dotenv()

True

In [4]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [5]:
# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [6]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    base_url="https://paper-api.alpaca.markets",
    api_version="v2")

In [7]:

def temp_test(tickers):
    
    #Obtaining todays date from system and formatting
    today = date.today()
    
    ####################
    #getting start date and end date for previous five years based on current year, i.e January 01 and December 31 for each year
    year1 = today.year - 4
    startdate_year1 = str(year1) + "-" + "01" + "-" + "01"
    enddate_year1 = str(year1) + "-" + "12" + "-" + "31"
    
    year2 = today.year - 3
    startdate_year2 = str(year2) + "-" + "01" + "-" + "01"
    enddate_year2 = str(year2) + "-" + "12" + "-" + "31"
    
    year3 = today.year - 2
    startdate_year3 = str(year3) + "-" + "01" + "-" + "01"
    enddate_year3 = str(year3) + "-" + "12" + "-" + "31"
    
    year4 = today.year - 1
    startdate_year4 = str(year4) + "-" + "01" + "-" + "01"
    enddate_year4 = str(year4) + "-" + "12" + "-" + "31"
    
    year5 = today.year
    startdate_year5 = str(year5) + "-" + "01" + "-" + "01"
    enddate_year5 = str(year5) + "-" + "12" + "-" + "31"
    ####################
         
    #converting dates to required format to pass to Alpaca API
    start_date1 = pd.Timestamp(startdate_year1, tz="America/New_York").isoformat()
    end_date1 = pd.Timestamp(enddate_year1, tz="America/New_York").isoformat()
    
    start_date2 = pd.Timestamp(startdate_year2, tz="America/New_York").isoformat()
    end_date2 = pd.Timestamp(enddate_year2, tz="America/New_York").isoformat()
    
    start_date3 = pd.Timestamp(startdate_year3, tz="America/New_York").isoformat()
    end_date3 = pd.Timestamp(enddate_year3, tz="America/New_York").isoformat()
    
    start_date4 = pd.Timestamp(startdate_year4, tz="America/New_York").isoformat()
    end_date4 = pd.Timestamp(enddate_year4, tz="America/New_York").isoformat()
    
    start_date5 = pd.Timestamp(startdate_year5, tz="America/New_York").isoformat()
    end_date5 = pd.Timestamp(enddate_year5, tz="America/New_York").isoformat()

    #####################
    
    # splitting the user selected company name to obtain the ticker. The ticker is a substring in the user selected company name
    temp_text = tickers
    stock_ticker = temp_text.split('Ticker: ', 1)[1].split(' ,',1)[0]
    
    ####################

    # Set timeframe to '1D' for Alpaca API
    timeframe = "1D"
    
    ####################
    # Get data for the tciker from alpaca for thye previous five years
    
    df_ticker1 = alpaca.get_barset(
    stock_ticker,
    timeframe,
    start=start_date1,
    end=end_date1,
    limit = 1000
     ).df
    
    df_ticker2 = alpaca.get_barset(
    stock_ticker,
    timeframe,
    start=start_date2,
    end=end_date2,
    limit = 1000
     ).df
    
    df_ticker3 = alpaca.get_barset(
    stock_ticker,
    timeframe,
    start=start_date3,
    end=end_date3,
    limit = 1000
     ).df
    
    df_ticker4 = alpaca.get_barset(
    stock_ticker,
    timeframe,
    start=start_date4,
    end=end_date4,
    limit = 1000
     ).df
    
    df_ticker5 = alpaca.get_barset(
    stock_ticker,
    timeframe,
    start=start_date5,
    end=end_date5,
    limit = 1000
     ).df
    
    # drop any records which are empty
    df_ticker1_clean = df_ticker1.dropna()
    df_ticker2_clean = df_ticker2.dropna()
    df_ticker3_clean = df_ticker3.dropna()
    df_ticker4_clean = df_ticker4.dropna()
    df_ticker5_clean = df_ticker5.dropna()
    
    # remove onel level of row heading from the alapca data
    df_ticker1_clean.columns = df_ticker1_clean.columns.droplevel()
    #compute percentage change in closing prices (daily returns)
    df_ticker1_clean['close'] = df_ticker1_clean['close'].pct_change()
    # drop the first row which is now NA after computing percentage change
    df_ticker1_clean = df_ticker1_clean.dropna()
    # ensure dataset has only daily returns and date which is an index
    df_ticker1_clean = df_ticker1_clean[['close']]
        
    #repeat below for all five years
    
    df_ticker2_clean.columns = df_ticker2_clean.columns.droplevel()
    df_ticker2_clean['close'] = df_ticker2_clean['close'].pct_change()
    df_ticker2_clean = df_ticker2_clean.dropna()
    df_ticker2_clean = df_ticker2_clean[['close']]
   
    
    df_ticker3_clean.columns = df_ticker3_clean.columns.droplevel()
    df_ticker3_clean['close'] = df_ticker3_clean['close'].pct_change()
    df_ticker3_clean = df_ticker3_clean.dropna()
    df_ticker3_clean = df_ticker3_clean[['close']]
        
    df_ticker4_clean.columns = df_ticker4_clean.columns.droplevel()
    df_ticker4_clean['close'] = df_ticker4_clean['close'].pct_change()
    df_ticker4_clean = df_ticker4_clean.dropna()
    df_ticker4_clean = df_ticker4_clean[['close']]
        
    df_ticker5_clean.columns = df_ticker5_clean.columns.droplevel()
    df_ticker5_clean['close'] = df_ticker5_clean['close'].pct_change()
    df_ticker5_clean = df_ticker5_clean.dropna()
    df_ticker5_clean = df_ticker5_clean[['close']]
        
    #append all five year daily % returns with date to a single dataset
    final_append = df_ticker1_clean.append([df_ticker2_clean, df_ticker3_clean, df_ticker4_clean, df_ticker5_clean])
    #find minimum value of daily % return
    minvalue = final_append['close'].min()
    #find maximum value of daily % return
    maxvalue = final_append['close'].max()
    #plot line graph of daily % return against date with scale of y axis set between set between maximum and minimum value of daily % return
    ax_final = final_append.hvplot.line(xlabel = 'Date', ylabel = 'Daily Returns %', title = f'Daily return % from {year1} to {year5}', ylim = (minvalue, maxvalue))
    
    ##############################################################
    #obtain copy of alapca datset for each of the five years
    #compute cumulative returns for each year
    # obtain the year end value of the cumulative return for each year
    
    df_tick1_cumulative =  df_ticker1_clean.copy()
    df_tick1_cumulative['close'] = (1+df_tick1_cumulative['close']).cumprod()
    df_year1_cumulative_bar = df_tick1_cumulative.tail(1)
    
    df_tick2_cumulative =  df_ticker2_clean.copy()
    df_tick2_cumulative['close'] = (1+df_tick2_cumulative['close']).cumprod()
    df_year2_cumulative_bar = df_tick2_cumulative.tail(1)
    
    df_tick3_cumulative =  df_ticker3_clean.copy()
    df_tick3_cumulative['close'] = (1+df_tick3_cumulative['close']).cumprod()
    df_year3_cumulative_bar = df_tick3_cumulative.tail(1)
    
    df_tick4_cumulative =  df_ticker4_clean.copy()
    df_tick4_cumulative['close'] = (1+df_tick4_cumulative['close']).cumprod()
    df_year4_cumulative_bar = df_tick4_cumulative.tail(1)
    
    df_tick5_cumulative =  df_ticker5_clean.copy()
    df_tick5_cumulative['close'] = (1+df_tick5_cumulative['close']).cumprod()
    df_year5_cumulative_bar = df_tick5_cumulative.tail(1)
    
    ####################################################################
    
    #append all year end cumulative return for each of the five years into a single dataset    
    bar_final = df_year1_cumulative_bar.append([df_year2_cumulative_bar, df_year3_cumulative_bar, df_year4_cumulative_bar, df_year5_cumulative_bar])
    bar_final.rename(columns={'close':'Cumulative Return'}, inplace = True)
    #plot the bar graph for cumulative return for each year
    ax_bar = bar_final.hvplot.bar(xlabel = 'Date', ylabel = 'Cumulative Annual Returns %', title = 'Annual Cumulative Returns for 5 years')
    
    
    #####################
    #line chart for cumulative returns
    
    #obtain new copy of daily returns % for each year
    df_tick11_cumulative =  df_ticker1_clean.copy()
    df_tick22_cumulative =  df_ticker2_clean.copy()
    df_tick33_cumulative =  df_ticker3_clean.copy()
    df_tick44_cumulative =  df_ticker4_clean.copy()
    df_tick55_cumulative =  df_ticker5_clean.copy()
    
    #combine all five year daily returns % into a single dataset
    line_temp = df_tick11_cumulative.append([df_tick22_cumulative,df_tick33_cumulative,df_tick44_cumulative,df_tick55_cumulative])
    #compute the cumulative returns across five years
    line_temp['close'] = (1+line_temp['close']).cumprod()
    # obtain name of the company based on the value selected by user
    title_name = tickers.split('Name: ',1)[1]
    #obtain line graph of cumulative returns across five years
    line_final = line_temp.hvplot.line(xlabel = 'Date', ylabel = 'Cumulative Returns %', title = f'Cumulative Returns for {title_name}')
    ##################
    
    #line graph of cumulative returns across five years and bar grpah of cumulative returns for each of the five years in one column
    final_out = pn.Column(line_final,ax_bar)
     
    #########################################################
       
    
    #get sp500 data, IVV is an SP500 ETF
    stock_ticker2 = "IVV"
    df_sp500 = alpaca.get_barset(
    stock_ticker2,
    timeframe,
    start=start_date1,
    end=end_date5,
    limit = 1000
     ).df
    
    df_sp500.columns = df_sp500.columns.droplevel()
    df_sp500 = df_sp500[['close']]
    df_sp500 = df_sp500.dropna()
    df_sp500.rename(columns={'close':'sp500proxy'}, inplace = True)
    #get daily % returns for sp500
    df_sp500['sp500proxy'] = df_sp500['sp500proxy'].pct_change()
    df_sp500 = df_sp500.dropna()
    ###############################################################
    
    final_append.rename(columns={'close':f'{stock_ticker}_returns'}, inplace = True)  
    #create new dataset with user selected ticker daily returns and sp500 returns    
    result = pd.concat([final_append, df_sp500], axis = 1, join = 'inner')
  
    ##################################################
    #Get Volatility of ticker and compare to SP500 Volatility
    
    #get copy of dataset with both ticker returns and sp500 returns
    result_dev = result.copy()
    result_dev.reset_index(inplace = True)
    result_dev['time'] = pd.to_datetime(result_dev['time']).dt.date
    result_dev.set_index('time')
    #obtain standard deviation of both ticker daily returns and sp500 daily returns----pass these to alist
    daily_standard_dev = result_dev.std().to_list()
    
    #convert the standard deviation of ticker and sp500 to annual form and pass to seprate variables
    stock_annual_dev = daily_standard_dev[0]*(252**(1/2))
    sp500_annual_dev = daily_standard_dev[1]*(252**(1/2))
    
    standard_deviation1 = None
    # create nbew dataset with only annualized standard deviation of sp500 and ticker
    standard_deviation1 = {f'{stock_ticker}_Volatility':[stock_annual_dev], 'SP500_Volatility': [sp500_annual_dev]}
    standard_deviation = pd.DataFrame(standard_deviation1)
    
    deviation_bar = None
    #get bar grpah of ticker and sp500 volatility
    deviation_bar = standard_deviation.hvplot.bar(ylabel = 'Volatility', title = f'Annual Volatility for {title_name} compared to SP500')
    ####################################################################
    #compute sharpe ratio for ticker using sp500 as benchmark
    
    result_sharpe = result.copy()
    sharpe_ratio = ((result_sharpe.mean()-result_sharpe['sp500proxy'].mean())*252)/((result_sharpe.std())*(252**(1/2)))
    sharpe_list = sharpe_ratio.to_list()
    sharpe_comment = f'The Sharpe Ratio for the company {title_name} is {round(sharpe_list[0],2)}'
    sharpe_graph1 = {f'Sharpe Ratio for {title_name}':[round(sharpe_list[0],2)]}
    sharpe_graph2 = pd.DataFrame(sharpe_graph1)
    #plot the sharpe ratio as a bar graph
    sharpe_graph = sharpe_graph2.hvplot.bar(title = f'The Sharpe Ratio for the company {title_name} is {round(sharpe_list[0],2)}' )
    
    ###################################################################
    #create required format
    
    final_out2 = pn.Column(deviation_bar,sharpe_graph)    
    final_out3 = pn.Row(final_out, final_out2)
    
    
    return(final_out3)

#######################################################################################################################################



In [8]:
# Get list of all tickers from Alpaca into stock_list
active_assets = alpaca.list_assets(status='active', asset_class = 'us_equity')

templist = []
#create list of alpaca assets in required format
for asset in active_assets:
    temp_var = asset.symbol + ' , Name: ' + asset.name
    templist.append(temp_var)

templist.sort()

stock_list = []
#add the word ticker to each alpaca asset in the list
for i in templist:
    temp_var = 'Ticker: ' + i
    stock_list.append(temp_var)
    
aaa = stock_list[9]
#testing if split works
aaa.split('Name: ',1)[1]

'AdvisorShares Dorsey Wright ADR ETF'

In [9]:
choose_stock = interact(temp_test, tickers = stock_list)
choose_stock

Column
    [0] Column
        [0] Select(name='tickers', options=['Ticker: A , ...], value='Ticker: A , N...)
    [1] Row
        [0] Row
            [0] Column
                [0] HoloViews(Curve)
                [1] HoloViews(Bars)
            [1] Column
                [0] HoloViews(Bars)
                [1] HoloViews(Bars)

In [10]:
# get Energy, Airline and Automobile performance assessment based on various ETF tickers
Energy_Secrtor = temp_test('Ticker: RYE ,Name: Energy ETF Proxy ticker - RYE')
Airlines_Sector = temp_test('Ticker: JETS ,Name: Airlines ETF Proxy ticker - JETS')
Automobile_Sector = temp_test('Ticker: CARZ ,Name: Automobiles ETF Proxy ticker - CARZ')

#create the first part of the final dashboard
final_dashboard1 = pn.Tabs(
    ("Energy Sector Performance", Energy_Secrtor),
    ("Airlines Performance", Airlines_Sector),
    ("Automobile Sector Performance", Automobile_Sector),
    ("Returns for Selected Stock", choose_stock),
                         )
#check if part of dashboar works
final_dashboard1

Tabs
    [0] Row
        [0] Column
            [0] HoloViews(Curve)
            [1] HoloViews(Bars)
        [1] Column
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
    [1] Row
        [0] Column
            [0] HoloViews(Curve)
            [1] HoloViews(Bars)
        [1] Column
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
    [2] Row
        [0] Column
            [0] HoloViews(Curve)
            [1] HoloViews(Bars)
        [1] Column
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
    [3] Column
        [0] Column
            [0] Select(name='tickers', options=['Ticker: A , ...], value='Ticker: A , N...)
        [1] Row
            [0] Row
                [0] Column
                    [0] HoloViews(Curve)
                    [1] HoloViews(Bars)
                [1] Column
                    [0] HoloViews(Bars)
                    [1] HoloViews(Bars)

In [11]:
# Reading the Mapbox API key from .env file.

map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)


In [12]:
#read US Pandemic data from csv file --- Ensure to CHANGE PATH HERE to the one used in your system for the source file
file_path = Path(r"C:\Users\Sobha\Desktop\Uoft\Homework\Project1\jack\us_data\us_data.csv")
us_data = pd.read_csv(file_path)
us_data[['date']] = us_data[['date']].apply(pd.to_datetime)
us_data.head()

,date,state Code,state_name,LAT,LONG,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,2020-01-22,NY,New York,43.00,-75.00,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Not agree,Not agree
1,2020-01-22,MP,Northern Mariana Islands,15.19,145.75,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Agree,Agree
2,2020-01-22,OK,Oklahoma,36.08,-96.92,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Not agree,Agree
3,2020-01-22,TN,Tennessee,35.86,-86.66,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Agree,Agree
4,2020-01-22,AR,Arkansas,34.80,-92.20,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Not agree,Not agree


In [13]:
date_list = us_data["date"].to_list()
date_list = list(dict.fromkeys(date_list))

#from the csv file obtain the unique list of US state names
state_list = us_data.sort_values("state_name", ascending=True)
state_list = state_list["state_name"].to_list()
state_list = list(dict.fromkeys(state_list))
state_list

['Alabama',
 'Alaska',
 'American Samoa',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'Federated States of Micronesia',
 'Florida',
 'Georgia',
 'Guam',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'New York City',
 'North Carolina',
 'North Dakota',
 'Northern Mariana Islands',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Palau',
 'Pennsylvania',
 'Puerto Rico',
 'Republic of Marshall Islands',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virgin Islands',
 'Virginia',
 'Washington',
 'Washington DC',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

In [14]:
def daily_cases(date):
    #create function to plot daily cumulative pandemic cases across each state
    day_plot = us_data.loc[us_data["date"] == date]
        
    date_plot = px.scatter_mapbox(
        data_frame=day_plot,
        hover_name="state_name",
        lat="LAT",
        lon="LONG",
        size="tot_cases",
        color="tot_cases",
        color_continuous_scale=px.colors.sequential.Jet,
        title="Cases Per region",
        zoom=2.3,
        width=1000,
        height=600)
    return date_plot 

#interact function to show map of US Pandemic cases
interact(daily_cases,date=date_list)

Column
    [0] Column
        [0] Select(name='date', options=[Timestamp('2020-01-22 00:...], value=Timestamp('2020-01-22 0...)
    [1] Row
        [0] Plotly(Figure, name='interactive04678')

In [15]:
us_data.head()

,date,state Code,state_name,LAT,LONG,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,2020-01-22,NY,New York,43.00,-75.00,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Not agree,Not agree
1,2020-01-22,MP,Northern Mariana Islands,15.19,145.75,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Agree,Agree
2,2020-01-22,OK,Oklahoma,36.08,-96.92,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Not agree,Agree
3,2020-01-22,TN,Tennessee,35.86,-86.66,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Agree,Agree
4,2020-01-22,AR,Arkansas,34.80,-92.20,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,3/26/2020 16:22,Not agree,Not agree


In [16]:
us_data_2 = us_data.drop(us_data.columns[[3,4,6,7,8,9,10,11,12,13,14,15,16,17]],axis=1)

us_data_2.head()

,date,state Code,state_name,tot_cases
0,2020-01-22,NY,New York,0
1,2020-01-22,MP,Northern Mariana Islands,0
2,2020-01-22,OK,Oklahoma,0
3,2020-01-22,TN,Tennessee,0
4,2020-01-22,AR,Arkansas,0


In [17]:
total_cases = us_data_2.groupby('date').sum()
total_cases= total_cases.sort_values(by="date",ascending=True)
total_cases.tail()

,tot_cases
date,
2021-04-01,30358137
2021-04-02,30424711
2021-04-03,30493173
2021-04-04,30533883
2021-04-05,30596830


In [18]:
total_cases.hvplot.bar(title = "Cumulative Pandemic cases in the US from 22 January 2020 to 05 April 2021", rot=90,height=600,width = 1000, xaxis = None).opts(yformatter="%.0f")

:Bars   [date]   (tot_cases)

In [19]:
state_plot = us_data_2.loc[us_data_2["state_name"] == "New York"]
state_plot.head()

,date,state Code,state_name,tot_cases
0,2020-01-22,NY,New York,0
116,2020-01-23,NY,New York,0
122,2020-01-24,NY,New York,0
239,2020-01-25,NY,New York,0
247,2020-01-26,NY,New York,0


In [20]:
#interact to plot cumulative pandemic cases per state
def state_cases(state):
    state_plot = us_data_2.loc[us_data_2["state_name"] == state]
    return state_plot.hvplot.bar(x="date", y="tot_cases", xaxis = None, title = "Cumulative Pandemic cases per state in the US from 22 January 2020 to 05 April 2021" ,rot=90,height=600,width = 1000).opts(yformatter="%.0f")

interact(state_cases,state=state_list)

Column
    [0] Column
        [0] Select(name='state', options=['Alabama', 'Alaska', ...], value='Alabama')
    [1] Row
        [0] HoloViews(Bars, name='interactive04880')

In [21]:
new_daily_cases =  us_data.drop(us_data.columns[[3,4,5,6,7,9,10,11,12,13,14,15,16,17]],axis=1)


In [22]:
def new_cases_per_state(state):
    
    new_daily_cases = us_data.loc[us_data["state_name"] == state]
    new_daily_cases = new_daily_cases.sort_values("date", ascending=True)
    return new_daily_cases.hvplot.bar(xaxis = None, x="date", y="new_case", title = "New Pandemic cases per state in the US from 22 January 2020 to 05 April 2021", rot=90,height=600,width = 1000).opts(yformatter="%.0f")


interact(new_cases_per_state,state=state_list)

Column
    [0] Column
        [0] Select(name='state', options=['Alabama', 'Alaska', ...], value='Alabama')
    [1] Row
        [0] HoloViews(Bars, name='interactive05080')

In [23]:
total_new_daily_cases = new_daily_cases.groupby("date").sum()
total_new_daily_cases.hvplot.bar(xaxis = None,  title = "New Pandemic cases in the US per day from 22 January 2020 to 05 April 2021" ,x="date", y="new_case",rot=90,height=600,width = 1300).opts(yformatter="%.0f")

:Bars   [date]   (new_case)

In [24]:
total_new_daily_cases.head()

,new_case
date,
2020-01-22,1
2020-01-23,0
2020-01-24,1
2020-01-25,0
2020-01-26,3


In [25]:
def regression_cases(tickers):    
    
    #regression against pandemic cases
      
    ################
    #obtain Ticker from user selected value
    temp_text = None    
    temp_text = tickers
    stock_ticker = None
    stock_ticker = temp_text.split('Ticker: ', 1)[1].split(' ,',1)[0]
    ################
    
    #get ticker data from January 2020 till 10 April 2021 which is in line with the pandemic data timeframe
    start_date1 = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()
    end_date1 = pd.Timestamp("2021-04-10", tz="America/New_York").isoformat()
    
    # Set timeframe to '1D' for Alpaca API
    timeframe = "1D"
    
    df_ticker1 = None
    df_ticker1 = alpaca.get_barset(
    stock_ticker,
    timeframe,
    start=start_date1,
    end=end_date1,
    limit = 1000
     ).df
    
    ################
    
    #retain only date and close price from ticker data
    df_ticker1.reset_index(inplace = True)
    df_ticker1.columns = df_ticker1.columns.droplevel()
    df_ticker1 = df_ticker1.rename(columns = {'':'date'})
    df_ticker1 = df_ticker1[['date','close']]
    
    ################
    
    #from the datetime in the column date, obtain only the date portion and exclude time
    df_ticker1['date'] = df_ticker1['date'].astype(str)
    df_ticker1['date'] = df_ticker1['date'].str.slice(0,10)
    
    #obtain copy of pandemic cases which was previously loaded outside of this function
    total_new_daily_cases2 = total_new_daily_cases.copy()
    total_new_daily_cases2.reset_index(inplace = True)
    total_new_daily_cases2['date'] = total_new_daily_cases2['date'].astype(str)
    
    #set date as index for both ticker data and pandemic data
    total_new_daily_cases2.set_index('date', inplace = True)
    df_ticker1.set_index('date', inplace = True)
    
    df_ticker1_clean = None
    df_ticker1_clean = df_ticker1.dropna()
    
    #obtain daily stock price returns for user selected ticker
    df_ticker1_clean['close'] = df_ticker1_clean['close'].pct_change()
    df_ticker1_clean = df_ticker1_clean.dropna()
    
    #obtain cumulative returns for user selected ticker
    df_tick1_cumulative =  df_ticker1_clean.copy()
    df_tick1_cumulative['close'] = (1+df_tick1_cumulative['close']).cumprod()
    #df_year4_cumulative_bar = df_tick4_cumulative.tail(1)
    
    
    #create final regression dataset of daily ticker price returns and new pandemic cases by inner join on date
    regression_data = pd.concat([df_ticker1_clean, total_new_daily_cases2], axis=1, join="inner")
    regression_data.reset_index(inplace = True)
    regression_data2 = regression_data.copy()
    
    regression_data = regression_data.dropna()
    
    #check regression dataset in csv format. Uncomment below code and provide relevant path in your system to download csv file
    #regression_data.to_csv(r'C:\Users\Sobha\Desktop\Uoft\Homework\Project1\jack\us_data\test.csv')
    
    #get both daily ticker price returns and new pandemic cases into separate lists
    y_reg = regression_data['close'].to_list()
    x_reg = regression_data['new_case'].to_list()
    
    #perform regression by passing above two lists
    result_reg = stats.linregress(x_reg, y_reg)
    
    #obtain regression results and pass relevant valuyes to separate strings for printing
    regression_equation_text = None
    result_reg2 = str(result_reg[0])
    result_reg3 = str(result_reg[1])
    regression_equation_text = "Daily % change in Closing Price = " + f"{result_reg2}*new_pandemic_cases + " + result_reg3
    
    #check if R square is greater than 505 or not. Print paapripriate message based on the check.
    flag = None
    if result_reg[2] > 0.5:
        flag = 1
    else:
        flag = 0
    
    r_value = None
    if flag == 1:
        r_value = "R-Square value (Multiple R Value) is " + str(result_reg[2]) + '. \nAs the R Square value is greater than 50%, the regression has acceptable predictive power'
    else:
        r_value = "R-Square value (Multiple R Value) is " + str(result_reg[2]) + '. \nAs the R Square value is less than 50%, the regression may not have acceptable predictive power'
        
    
    #correlation_price_cases = regression_data2['close'].corr(regression_data2['new_case'])
    #correlation_text = "The correlation between the % change in closing price and new pandemic cases is " + str(correlation_price_cases)
    
    #plot line chart of new daily pandemic cases by CAPPING the chart dimension between 0 and maximum value of ticker daily percentage return
    regression_data3 = regression_data.copy()
    regression_data3['new_case'] = ((regression_data3['new_case']-regression_data3['new_case'].min())/(regression_data3['new_case'].max()-regression_data3['new_case'].min()))*regression_data3['close'].max()
    line_chart1 = None
    line_chart1 = regression_data3.hvplot.line(y = ['new_case', 'close'], x= 'date', ylabel="Daily Pandemic cases Normalized to maximum percentage change in stock price",title = f"Daily % change in closing prices of ticker {stock_ticker} compared to Daily Pandemic cases in the US", xaxis = None, height = 700, width = 1000)
    
    #plot line chart of new daily pandemic cases by CAPPING the chart dimension between 0 and maximum value of ticker cumulative percentage return
    regression_data4 = pd.concat([df_tick1_cumulative, total_new_daily_cases2], axis=1, join="inner")
    regression_data4['new_case'] = ((regression_data4['new_case']-regression_data4['new_case'].min())/(regression_data4['new_case'].max()-regression_data4['new_case'].min()))*regression_data4['close'].max()
    line_chart2 = None
    line_chart2 = regression_data4.hvplot.line(y = ['new_case', 'close'], x = 'date', ylabel="Daily Pandemic cases Normalized to maximum cumulative percentage change in stock price",title = f"Cumulative % change in closing prices of ticker {stock_ticker} compared to Daily Pandemic cases in the US", xaxis = None, height = 700, width = 1000)
    
    title_name = None
    title_name = tickers.split('Name: ',1)[1]
    
    #create required layout for data presentation    
    regression_output1 = pn.Column(regression_equation_text, r_value,line_chart1)
    regression_output2 = pn.Row(regression_output1,line_chart2)
    regression_output3 = pn.Column(f'#Output of Regression - % Daily Change in Closing Prices of {title_name} compared to daily Pandemic Cases',result_reg, regression_output2)
    
    return(regression_output3)#regression_output)
    

In [26]:
interact(regression_cases, tickers = stock_list)
#stock_list

Column
    [0] Column
        [0] Select(name='tickers', options=['Ticker: A , ...], value='Ticker: A , N...)
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] Str(LinregressResult)
            [2] Row
                [0] Column
                    [0] Markdown(str)
                    [1] Markdown(str)
                    [2] HoloViews(NdOverlay)
                [1] HoloViews(NdOverlay)

In [27]:
#create final dashboard for presentation

cases_per_state = interact(new_cases_per_state,state=state_list)
cases_in_country = total_new_daily_cases.hvplot.bar(xaxis = None,  title = "New Pandemic cases in the US per day from 22 January 2020 to 05 April 2021" ,x="date", y="new_case",rot=90,height=600,width = 1000).opts(yformatter="%.0f")
cases_incountry_map = interact(daily_cases,date=date_list)
map_of_cases1 = pn.Column(cases_per_state, cases_in_country)
map_of_cases2 = pn.Row(cases_incountry_map, map_of_cases1)

regression_tab = interact(regression_cases, tickers = stock_list)

final_dashboard1 = None

final_dashboard1 = pn.Tabs(
    ("Energy Sector Performance", Energy_Secrtor),
    ("Airlines Performance", Airlines_Sector),
    ("Automobile Sector Performance", Automobile_Sector),
    ("Returns for Selected Stock", choose_stock),
    ("Pandemic Cases", map_of_cases2),
    ("Regression Analysis of Pandemic", regression_tab)
                         )
final_dashboard1

Tabs
    [0] Row
        [0] Column
            [0] HoloViews(Curve)
            [1] HoloViews(Bars)
        [1] Column
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
    [1] Row
        [0] Column
            [0] HoloViews(Curve)
            [1] HoloViews(Bars)
        [1] Column
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
    [2] Row
        [0] Column
            [0] HoloViews(Curve)
            [1] HoloViews(Bars)
        [1] Column
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
    [3] Column
        [0] Column
            [0] Select(name='tickers', options=['Ticker: A , ...], value='Ticker: A , N...)
        [1] Row
            [0] Row
                [0] Column
                    [0] HoloViews(Curve)
                    [1] HoloViews(Bars)
                [1] Column
                    [0] HoloViews(Bars)
                    [1] HoloViews(Bars)
    [4] Row
        [0] Column
            [0] Column
                [0] Select(name='date', options=[Timestamp('2020-01-22 00:...], value=Timestamp('2020-01-22 0...)
            [1] Row
                [0] Plotly(Figure, name='interactive06667')
        [1] Column
            [0] Column
                [0] Column
                    [0] Select(name='state', options=['Alabama', 'Alaska', ...], value='Alabama')
                [1] Row
                    [0] HoloViews(Bars, name='interactive06406')
            [1] HoloViews(Bars)
    [5] Column
        [0] Column
            [0] Select(name='tickers', options=['Ticker: A , ...], value='Ticker: A , N...)
        [1] Row
            [0] Column
                [0] Markdown(str)
                [1] Str(LinregressResult)
                [2] Row
                    [0] Column
                        [0] Markdown(str)
                        [1] Markdown(str)
                        [2] HoloViews(NdOverlay)
                    [1] HoloViews(NdOverlay)